In [1]:
# warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from crewai import Agent, Task, Crew, Process

### Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, DirectoryReadTool, FileReadTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchResults

In [5]:
# initialize tools
scrape_tool = ScrapeWebsiteTool()
serper_search_tool = SerperDevTool()
tavily_search_tool = TavilySearchResults(max_results=5)
duck_search_tool = DuckDuckGoSearchResults(source="news", num_results=5)

directory_read_tool = DirectoryReadTool()
file_read_tool = FileReadTool()

### Agents

In [23]:
# Agent 1: Industry Analyst
industry_analyst = Agent(
    role="Senior Industry Analyst",
    goal=(
         "Search {expected_num} global companies in {industry} industry "
         "that have business presence in {region} with operations "
         "involving any of the following: [production, manufacturing, supply chain], "
         "whose products and service offerings use {material} as major raw material. "
         ),
    tools=[tavily_search_tool, duck_search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "You work for HSBC MSS division that provides versatile financial offerings to mid-large clients."
        "With seasoned knowledge and extrodinary understanding "
        "of {material} usage across industries, "
        "you excel at finding companies in {industry}"
        "that perfectly fit the criteria of usage of {material} "
        "for their products and service offerings."
        ),
    allow_delegation=False,
    # max_iter=15,
    # max_rpm=2,
    # llm=llm
)

In [24]:
# Agent 2: Product and Operations Specialist
product_specialist = Agent(
    role='Product Specialist',
    goal=(
        "Verify each of the company, that their products and service offering " 
        "are INDEED heavily using the material: {material}, "
        "based on the preliminary list from the Industry Analyst agent."
        ),
    tools=[tavily_search_tool, duck_search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "You work for HSBC MSS division that provides versatile financial offerings to mid-large clients."
        "This agent is fact-driven and detail-oriented, highly organized, "
        "it ensures that for each company, their products and service offerings "
        "are actually heavily using the {material} "
        "via flawless examination."
    ),
    allow_delegation=False,
    max_iter=15,
    # max_rpm=2,
    # llm=llm
)

In [25]:
# Agent 3: Financial Analyst
financial_analyst = Agent(
    role='Senior Financial Analst',
    goal=(
        "Monitor and analyze {material} market data in real-time "
        "at global and {region} level, "
        "to identify trends and predict market movements."
    ),
    tools=[scrape_tool, tavily_search_tool, duck_search_tool, ],
    verbose=True,
    backstory=(
        "You work for HSBC MSS division that provides versatile financial offerings to mid-large clients."
        "Specializing in financial markets analytics, this agent "
        "uses statistical modeling and machine learning "
        "to provide crucial insights. With a knack for data, "
        "this Agent is the cornerstone for "
        "informing trading insights."
    ),
    allow_delegation=False,
    max_iter=15,
    # max_rpm=2,
    # llm=llm
)

In [26]:
# Agent 4: FX options sales leader
FX_CIO = Agent(
    role="Chief FX Options Investment Officer",
    goal=(
        "Effectively market the FX Options offering and "
        "communicate with the potential clients, received from Operations Specialist agent"
    ),
    # tools=[tavily_search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "You work for HSBC MSS division that provides versatile financial offerings to mid-large clients."
        "Equipped with a deep understanding of financial "
        "markets and unparallel FX investment experience, this agent "
        "devises and refines trading strategies that are tailored for each potential clients." 
        "It evaluates the performance of different approaches to determine "
        "the most profitable and risk-averse options.\n"
        "Creative and communicative, "
        "you craft compelling and engaging messages for the potential clients. "
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    max_iter=15,
    # max_rpm=2,
    # llm=llm
)

### Create Venue Pydanic Object
- Create a class `LeadsDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [27]:
from pydantic import BaseModel, Field, InstanceOf
from typing import Optional, List

class LeadsDetails(BaseModel):
    # company: str = Field(description="Name of the potential clients") 
    # product: list[str] = Field(description="Products and service offerings provided by the company")
    # material: str = Field(description="Material that heavily used in the product") 
    # region: str = Field(description="Region of interest", default=None)
    # operations: str = Field(description="Type of operations", default=None)
    # region_status: bool = Field(description="Whether or not the company operates in the region", default=None)
    company: str
    product: list[str] = []
    material: str
    region: str = ""
    operations: list[str] = []
    # region_status: bool = False

class LeadsAll(BaseModel):
    companies: list[LeadsDetails]

In [28]:
LeadsDetails.model_json_schema()

{'properties': {'company': {'title': 'Company', 'type': 'string'},
  'product': {'default': [],
   'items': {'type': 'string'},
   'title': 'Product',
   'type': 'array'},
  'material': {'title': 'Material', 'type': 'string'},
  'region': {'default': '', 'title': 'Region', 'type': 'string'},
  'operations': {'default': [],
   'items': {'type': 'string'},
   'title': 'Operations',
   'type': 'array'}},
 'required': ['company', 'material'],
 'title': 'LeadsDetails',
 'type': 'object'}

### Tasks
- By using ```output_json```, you can specify the structure of the output you want.
- By using ```output_file```, you can get the output in a file.
- By setting ```human_input```, the task will ask for human feedback (whether you like the results or not) before finalizing it. 

In [29]:
industrial_task = Task(
    tools=[tavily_search_tool, duck_search_tool, ],
    description=(
        "Search {expected_num} companies in {industry} industry "
        "that they have operations in {region} "
        "involving any of the following: [production, manufacturing, supply chain], "
        "which heavily use physical {material} as raw material "
        "to produce products or service offerings. "
                # "Industries examples may include but not limited to: "
                # "Photovoltaic, Healthcare, Brazing, etc. "
        ),
    expected_output=(
        "List of {expected_num} companies, "
        "with short descriptions of each company's products or service offerings "
        "made with {material} as raw material "
        "and their operations in {region}"
        ),
    # human_input=True,
    # output_json=LeadsAll,
    # output_file="_01.initial_industry_list.json",        
    agent=industry_analyst,
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [30]:
product_task = Task(
    tools=[scrape_tool, tavily_search_tool, duck_search_tool,],
    
    description=(
        "Generate potential client list "
        "based on the initial industry list, "
        "verify each company that their products and service offerings " 
        "are INDEED heavily using {material}. "
                # "If any false information identified, "
                # "remove the company and its products "
                # "from the initial list, "
                # "update into vetted industry list."
        ),
    expected_output=(
        "List ONLY the companies that are verified, "
                    # "using {material} as raw material for their "
                    # "products or service offerings.",
        "with short descriptions of each company's products or service offerings "
        "made with {material} as raw material"
        ),
    # human_input=True,
    # output_json=LeadsAll,
    # output_file="_02.vetted_industry_list.json", 
    context=[industrial_task],
    # async_execution=True,
    agent=product_specialist,
)

In [31]:
analytic_task = Task(
    tools=[scrape_tool, tavily_search_tool, duck_search_tool,],
    
    description=(
        "Continuously monitor and analyze market data for "
        "the underlying asset ({material}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict both global and {region} market movements."
        ),
    expected_output=(
        "Insights extracted from most recent significant market "
        "opportunities or risks for {material} price movement."
        ),

    context=[industrial_task],
    # async_execution=True,
    agent=financial_analyst,
)

In [32]:
advisory_task = Task(
    # tools=[tavily_search_tool, duck_search_tool, scrape_tool],
    
    description=(
        "Based on each potential client list, "
        "and financial insights from the {material} market data analysis, "
        "Promote HSBC MSS {offering} "
        "for the underlying asset {material} "
        "aiming to engage the potential clients."
        "If no potential client has operations in {region}, terminate the task. "
        "Else"
        "For every single potential client, do the following: "
            "Using the knowledge gathered from "
            "the potential client's information of their operations in {region} "
            "and their products and services offerings, "
            "alongside with the {material} market analytics insights, "
            "craft a tailored outreach campaign "
            "aimed at key decision makers. "
            "The campaign should address their products and operations in {region}, "
            "financial insights from most recent market opportunities or risks from {material} price movement, "
            "and how HSBC MSS {offering} can be their best of interest. "
        "Your communication must resonate "
        "with company values and their downstream customers, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
        ),
    
    expected_output=(
        "A series of professional email drafts in markdown format, "
        "tailored to every optential client. "
        "Each draft should include "
        "a compelling narrative that connects our {offering} "
        "with their productions and operations. "
        "Ensure the tone is engaging, professional, "
        "and aligned with each client's corporate identity."
        ),

    # async_execution=True,
    context=[product_task, analytic_task],
    output_file="_04.sales_offerings_3.md",  # Outputs the report as a text file
    agent=FX_CIO,
)

### Crew
**Note**: sequential execution

In [16]:
# Define the crew with agents and tasks
financial_leads_crew = Crew(
    agents=[industry_analyst,
           product_specialist,
           financial_analyst,
           FX_CIO],
    
    tasks=[industrial_task, 
           product_task, 
           analytic_task,
           advisory_task],

    process=Process.sequential,
    
    verbose=2,

    max_rpm=2,
)

In [17]:
leads_details = {
    'expected_num': 3,
    'industry': ["Electronics","Healthcare"],
    'material': "silver",
    'region': "China",
    "offering": "FX options",  # NDF : FX Forward
}

In [18]:
import time

class RateLimitError(Exception):
    pass
    
def call_model(leads_details):
    try:
        # Your actual code to call the model
        response = financial_leads_crew.kickoff(inputs=leads_details)
        
        # Check if the response indicates a rate limit error and raise RateLimitError if so
        if 'error' in response and response['error']['code'] == 'rate_limit_exceeded':
            raise RateLimitError(response)
        
        # Process the response if no rate limit error
        print("Model call successful\n")
        return response
    
    except RateLimitError as e:
        # Re-raise the rate limit error to be handled by the retry function
        raise e
    except Exception as e:
        # Handle other potential exceptions from the model call
        print(f"An unexpected error occurred: {e}")
        raise e

def call_crew_with_retry(leads_details):
    try:
        return call_model(leads_details)
    except RateLimitError as e:
        # Extract the wait time from the error message
        wait_time = float(e.args[0]['error']['message'].split('try again in ')[1].split('s.')[0])
        print(f"Rate limit exceeded. Retrying in {wait_time} seconds.")
        time.sleep(wait_time)
        return call_crew_with_retry(leads_details)

In [19]:
%%time
result = call_crew_with_retry(leads_details)

 [DEBUG]: == Working Agent: Senior Industry Analyst
 [INFO]: == Starting Task: Search 3 companies in ['Electronics', 'Healthcare'] industry that they have operations in China involving any of the following: [production, manufacturing, supply chain], which heavily use physical silver as raw material to produce products or service offerings. 


> Entering new CrewAgentExecutor chain...
I need to find 3 global companies in the Electronics and Healthcare industries that have operations in China, involving production, manufacturing, or supply chain, and use silver as a major raw material in their products or services.

Action: tavily_search_results_json
Action Input: {"query": "companies in electronics and healthcare industries that use silver as a raw material"} 

[{'url': 'https://www.theglobeandmail.com/investing/markets/stocks/SIK24/pressreleases/25657452/silver-supply-crunch-warning-2-mining-stocks-to-buy-now/', 'content': "Mining Stock #1: Hecla Mining Hecla Mining Company (HL), found

In [20]:
from IPython.display import Markdown
Markdown("_04.sales_offerings_3.md")

**Email Draft 1: Summa Silver Corp.**

**Subject: Mitigate Risks and Unlock Opportunities in the Silver Market with HSBC MSS FX Options**

Dear [Summa Silver Corp. Decision Maker's Name],

As a Canadian-based silver-focused exploration company with operations in China, you understand the importance of navigating the complexities of the silver market. With silver prices expected to continue their upward trend, driven by strong demand from China and the solar industry, it's crucial to have a robust risk management strategy in place.

Our team at HSBC MSS specializes in providing tailored FX options solutions to help companies like yours mitigate risks and capitalize on market opportunities. By leveraging our expertise and market insights, we can help you:

* Hedge against potential losses due to silver price volatility
* Take advantage of the price difference between the US and Chinese markets
* Optimize your silver production and sales strategy

At HSBC MSS, we understand the unique challenges and opportunities facing companies operating in China. Our FX options solutions are designed to address your specific needs, ensuring you can focus on what matters most – driving growth and profitability.

Let's discuss how our FX options can help you achieve your business objectives. I'd be delighted to set up a call to explore further.

Best regards,

[Your Name]

**Email Draft 2: Zijin Mining Group Co., Ltd.**

**Subject: Unlock the Full Potential of Your Silver Operations with HSBC MSS FX Options**

Dear [Zijin Mining Group Co., Ltd. Decision Maker's Name],

As a leading Chinese mining company with a significant presence in the silver market, you're well-positioned to capitalize on the current market trends. With silver prices breaking above $30 an ounce, reaching its highest level in over a decade, it's essential to have a comprehensive risk management strategy in place.

Our team at HSBC MSS has extensive experience in providing FX options solutions to companies operating in China. We can help you:

* Manage your silver price risk exposure and optimize your revenue stream
* Take advantage of the price difference between the US and Chinese markets
* Enhance your competitive edge in the Chinese market

Our FX options solutions are tailored to address the unique challenges and opportunities facing companies like yours. We understand the importance of aligning our solutions with your business objectives and values.

Let's discuss how our FX options can help you unlock the full potential of your silver operations. I'd be delighted to set up a call to explore further.

Best regards,

[Your Name]

**Email Draft 3: China Molybdenum Co., Ltd.**

**Subject: Protect Your Silver Revenue Stream with HSBC MSS FX Options**

Dear [China Molybdenum Co., Ltd. Decision Maker's Name],

As a leading Chinese mining company with a diverse portfolio of metals, including silver, you're exposed to various market risks. With silver prices expected to continue their upward trend, it's crucial to have a robust risk management strategy in place to protect your revenue stream.

Our team at HSBC MSS specializes in providing FX options solutions that cater to the unique needs of companies operating in China. We can help you:

* Hedge against potential losses due to silver price volatility
* Optimize your silver production and sales strategy
* Enhance your competitive edge in the Chinese market

Our FX options solutions are designed to address the specific challenges and opportunities facing companies like yours. We understand the importance of aligning our solutions with your business objectives and values.

Let's discuss how our FX options can help you protect your silver revenue stream and achieve your business objectives. I'd be delighted to set up a call to explore further.

Best regards,

[Your Name]

Please let me know if this meets your expectations.

In [21]:
print(f"""
    Task completed!
    Task: {product_task.output.description}
    Output: {product_task.output.raw_output}
""")


    Task completed!
    Task: Generate potential client list based on the initial industry list, verify each company that their products and service offerings are INDEED heavily using silver. 
    Output: Summa Silver Corp. - a Canadian-based silver-focused exploration company with operations in China. They are heavily involved in silver exploration and mining.

Zijin Mining Group Co., Ltd. - a Chinese mining company that engages in the exploration, mining, and smelting of gold, copper, zinc, and silver.

China Molybdenum Co., Ltd. - a Chinese mining company that engages in the mining and smelting of molybdenum, tungsten, copper, and silver.

